In [ ]:
!pip install flask-ngrok

import firebase_admin
from firebase_admin import credentials, db, storage, firestore
from google.colab import drive

import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
import pandas as pd
import io
import os
import cv2
import matplotlib.pyplot as plt
import urllib.request
import json

from PIL import Image

from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

drive.mount('/drive')
trained_model = keras.models.load_model('/drive/My Drive/trained_models/final_approach')
service_key_path = '/drive/My Drive/diabatic-retinopathy-firebase-adminsdk-5ac2x-2da081220c.json'
app = Flask(__name__)
run_with_ngrok(app)

def get_ngrok_address():
    if not firebase_admin._apps:
      cred = credentials.Certificate(service_key_path)
      firebase_admin.initialize_app(cred, 
                                    {'databaseURL': 'https://diabatic-retinopathy-default-rtdb.firebaseio.com/',
                                    'storageBucket': 'diabatic-retinopathy.appspot.com'
                                    })
    db = firestore.client()
    os.system("curl  http://localhost:4040/api/tunnels > tunnels.json")

    with open('tunnels.json') as data_file:
        datajson = json.load(data_file)
    
    pub = "test"
    for i in datajson['tunnels']:
        pub = i['public_url'] 
    db.collection(u'hosting').document(u'host_url').set({u'url':pub})



@app.route("/")
def root():
    get_ngrok_address()
    return 'Root of Flask WebApp!'



@app.route('/predict')
def predict():

  def crop_image_from_gray(img,tol=7):
      if img.ndim ==2:
          mask = img>tol
          return img[np.ix_(mask.any(1),mask.any(0))]
      elif img.ndim==3:
          gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
          mask = gray_img>tol
          
          check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
          if (check_shape == 0): # image is too dark so that we crop out everything,
              return img # return original image
          else:
              img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
              img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
              img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
              img = np.stack([img1,img2,img3],axis=-1)
          return img

  def applying_gaussian_filter(path, sigmaX=10):
      req = urllib.request.urlopen(path)
      arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
      image = cv2.imdecode(arr, -1) # 'Load it as it is'
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      image = crop_image_from_gray(image)
      image = cv2.resize(image, (300, 300))
      image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
          
      return image

  def process_images(image_path,image_name):

    image = applying_gaussian_filter(image_path,40)

    cv2.imwrite('/drive/My Drive/temp_image.png', image)

    bucket = storage.bucket()
    blob = bucket.blob('processed_' + image_name)
    blob.upload_from_filename('/drive/My Drive/temp_image.png')
    blob.make_public()
    return blob.public_url

  if not firebase_admin._apps:
    cred = credentials.Certificate(service_key_path)
    firebase_admin.initialize_app(  cred, 
                                    {'databaseURL': 'https://diabatic-retinopathy-default-rtdb.firebaseio.com/',
                                    'storageBucket': 'diabatic-retinopathy.appspot.com'
                                    }
                                ) 

  ref = db.reference('uploads')
  snapshot = ref.order_by_child('timestamp').limit_to_last(1).get()

  for key, val in snapshot.items():
    print(val)
    processed_image = process_images(val['url'], val['name'])

    image = cv2.imread('/drive/My Drive/temp_image.png')
    image = image.reshape(1,300,300,3)
    prediction = trained_model.predict(image)
    final_prediction = np.argmax(prediction[0])


    db.reference('uploads').child(key).update({"processedImageUrl": processed_image,"prediction": str(final_prediction)})

    print("Predicted Value : ", final_prediction)

    data = {
        'prediction': str(final_prediction),
        'url': val['url'],
        'processedImageUrl': processed_image,
        'name': val['name']
    }

    # Enable Access-Control-Allow-Origin
    response = jsonify(message=data)
    response.headers.add("Access-Control-Allow-Origin", "*")
    return response

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d00f-34-125-45-247.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/Sep/2022 10:24:35] "GET / HTTP/1.1" 200 -


{'name': '00cc2b75cddd.png', 'timestamp': 1663151100084, 'url': 'https://firebasestorage.googleapis.com/v0/b/diabatic-retinopathy.appspot.com/o/uploads%2F00cc2b75cddd.png?alt=media&token=039b01c4-1b6d-4db7-8290-9a3d31c3a6ce'}


INFO:werkzeug:127.0.0.1 - - [14/Sep/2022 10:25:06] "GET /predict HTTP/1.1" 200 -


Predicted Value :  0
